In [1]:
import numpy as np
import pandas as pd

import IPython.display as ipd

import librosa
import librosa.display

import matplotlib.pyplot as plt

import cv2
%matplotlib inline
import numpy as np
import matplotlib
# prikaz vecih slika 
matplotlib.rcParams['figure.figsize'] = 16,12

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn import metrics

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, BatchNormalization, MaxPool2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint
from datetime import datetime

Using TensorFlow backend.


In [2]:
def extract_spectogram(file_name):
    try:
        audio, sample_rate = librosa.load(file_name,res_type="kaiser_fast" ,sr=22050)
        a, index = librosa.effects.trim(audio, top_db=30, frame_length=2048, hop_length=512)
        y_out = a[:44100]
        spectrogram = librosa.feature.melspectrogram(y=y_out, sr=sample_rate, n_fft=2048, hop_length=1024)
        #print(spectrogram)
        spec_shape = spectrogram.shape
        if(spec_shape[1] < 44):
            print(spec_shape)
            print(file_name)
        spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
        #imarray = np.reshape(spectrogram, (128, -1, 3))
        #img_gray = np.dot(spectrogram[...,:3], [0.299, 0.587, 0.114])
        #img_resized = cv2.resize(img_gray, (128,128), interpolation=cv2.INTER_CUBIC)
        #print(img_resized.shape)
        #redgram = block_reduce(img_gray, block_size = (3,3), func = np.mean)
        #return img_gray
        #mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        #mfccscaled = np.mean(mfccs.T, axis=0)
        return spectrogram
    
    except Exception as e:
        print(e)
        print("Error encountered while parsing file: ", file_name)
        return None
    
    return mfccscaled    

def my_rgb2gray(img_rgb):
    img_gray = np.ndarray((img_rgb.shape[0], img_rgb.shape[1]))  # zauzimanje memorije za sliku (nema trece dimenzije)
    img_gray = 0.21*img_rgb[:, :] + 0.72*img_rgb[:, :] #+ 0.07*img_rgb[:, :, 2]
#     img_gray = img_gray.astype('uint8')  # u prethodnom koraku smo mnozili sa float, pa sada moramo da vratimo u [0,255] opseg
    return img_gray

def normalize_gray(array):
    return (array - array.min())/(array.max() - array.min())

In [3]:
folder_path = "dataset/"
folder_list = ["Accordion","Clarinet_Bb","Contrabass","Horn","Viola","Violin","Violoncello"]
features = []
onlyfiles = []

from os import listdir
from os.path import isfile, join

for folder_inst in folder_list:
    onlyfiles = onlyfiles + [f for f in listdir("dataset/" + folder_inst) if isfile(join("dataset/" + folder_inst + "/", f))]
    


for file in onlyfiles: 
    dodatak = ''
    if (file.split("-")[0] == "Acc"):
        dodatak = "Accordion"
    elif (file.split("-")[0] == "ClBb"):
        dodatak = "Clarinet_Bb"
    elif (file.split("-")[0] == "Cb"):
        dodatak = "Contrabass"
    elif (file.split("-")[0] == "Hn"):
        dodatak = "Horn"
    elif (file.split("-")[0] == "Va"):
        dodatak = "Viola"
    elif (file.split("-")[0] == "Vn"):
        dodatak = "Violin"
    elif (file.split("-")[0] == "Vc"):
        dodatak = "Violoncello"
    
    file_name = folder_path + dodatak + "/" + file
    a = file.split("-")
    instrument = a[0]
    pitch = a[2]
    if(len(pitch) == 3):
        pitch = pitch[:2]
    else:
        pitch = pitch[:1]
    data = extract_spectogram(file_name)
    features.append([file_name, data, instrument, pitch])
    #print(img)
    

In [4]:
#menjanje boje slike
features_gray = []
for list in features:
    imggray = my_rgb2gray(list[1])
    features_gray.append([list[0],normalize_gray(imggray), list[2], list[3]])
    
features_df = pd.DataFrame(features_gray, columns=["file", "feature", "instrument", "pitch"])
features_df.head()

,file,feature,instrument,pitch
0,dataset/Accordion/Acc-ord-A#3-ff-alt1-N.wav,"[[0.29073387, 0.33191147, 0.2776536, 0.2996274...",Acc,A#
1,dataset/Accordion/Acc-ord-A#3-ff-alt2-N.wav,"[[0.3070023, 0.3617015, 0.27213687, 0.36119562...",Acc,A#
2,dataset/Accordion/Acc-ord-A#3-ff-N-N.wav,"[[0.47353348, 0.42426324, 0.3613015, 0.4068834...",Acc,A#
3,dataset/Accordion/Acc-ord-A#3-mf-alt1-N.wav,"[[0.42782307, 0.46233648, 0.38736117, 0.385382...",Acc,A#
4,dataset/Accordion/Acc-ord-A#3-mf-alt2-N.wav,"[[0.6518973, 0.62134737, 0.5658642, 0.48233718...",Acc,A#


In [5]:
X = np.array(features_df.feature.tolist())
yinst = np.array(features_df.instrument.tolist())

leinst = LabelEncoder()
yyinst = leinst.fit_transform(yinst)

x_train, x_test, y_train, y_test = train_test_split(X, yyinst, test_size=0.2, shuffle=True, random_state=42)
x_validation = x_train.reshape(len(x_train),128,44,1)
print(x_validation.shape)

x_test = x_test.reshape(len(x_test),128,44,1)
print(x_test.shape)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, shuffle=True, random_state=42)
x_val = x_val.reshape(len(x_val),128,44,1)
print(x_val)
print(x_test)

num_classes = len(leinst.classes_)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)

(1552, 128, 44, 1)
(388, 128, 44, 1)
[[[[0.50546366]
   [0.50976557]
   [0.45719093]
   ...
   [0.47455534]
   [0.4843364 ]
   [0.6485551 ]]

  [[0.4062378 ]
   [0.55645317]
   [0.4695128 ]
   ...
   [0.51963496]
   [0.48358884]
   [0.67521095]]

  [[0.42677134]
   [0.4729444 ]
   [0.45165122]
   ...
   [0.40431026]
   [0.41397977]
   [0.69928926]]

  ...

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.02933482]
   [0.08929642]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.07193135]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.07142293]]]


 [[[0.6268529 ]
   [0.5135684 ]
   [0.5240812 ]
   ...
   [0.4579794 ]
   [0.45289996]
   [0.80740666]]

  [[0.67740375]
   [0.72938544]
   [0.75785244]
   ...
   [0.8209817 ]
   [0.82015157]
   [0.79487294]]

  [[0.72822857]
   [0.73406047]
   [0.7645523 ]
   ...
   [0.82897407]
   [0.82896376]
   [0.7860463 ]]

  ...

  

In [35]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(11,11), strides=(1,1), activation='relu', input_shape=(128,44,1 )))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3,3), strides=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
#Compile
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 118, 34, 32)       3904      
_________________________________________________________________
batch_normalization_35 (Batc (None, 118, 34, 32)       128       
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 58, 16, 32)        0         
_________________________________________________________________
dropout_43 (Dropout)         (None, 58, 16, 32)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 58, 16, 64)        51264     
_________________________________________________________________
batch_normalization_36 (Batc (None, 58, 16, 64)        256       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 28, 7, 64)       

In [39]:
num_epochs = 60
batch_size = 32

checkpointer = ModelCheckpoint(filepath="best_weights_inst.hdf5", verbose=1, save_best_only=True)

start = datetime.now()
model.fit(x_validation, y_train, batch_size=batch_size, epochs=num_epochs, validation_data=(x_val, y_val), 
           callbacks=[checkpointer], verbose=1)

duration = datetime.now() - start
print("Training completed in: ", duration)

Train on 1552 samples, validate on 194 samples
Epoch 1/50
 160/1552 [==>...........................] - ETA: 45s - loss: 0.0381 - accuracy: 0.9875

KeyboardInterrupt: 

In [37]:
train_score = model.evaluate(x_validation, y_train, verbose=1)
print("Training Accuracy: ", train_score[1])

1552/1552 [==============================] - 12s 8ms/step
Training Accuracy:  0.9774484634399414


In [38]:
test_score = model.evaluate(x_test, y_test, verbose=1)
print("Test Accuracy: ", test_score[1])

194/194 [==============================] - 2s 8ms/step
Test Accuracy:  0.9484536051750183
